# Ähnlichkeitsmaße für Gesten

In [1]:
import os
import sys
import numpy as np
import scipy.spatial.distance as dis
import scipy.stats as st
import time

from glob import glob
from gesture_comparison_helper import load_gestures
from itertools import combinations, combinations_with_replacement
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Utils.data_preparation_helper import load_data_for_classification, prepare_data

c:\python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def calculate_similarity(a, b, sim_type):
    if sim_type == 'cosine':
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif sim_type == 'euclid':
        return 1/(1+dis.euclidean(a, b))
    elif sim_type == 'pearson':
        return st.pearsonr(a, b)[0]
    else:
        print('sim_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(sim_type))

def calculate_distance(a, b, dist_type):
    if dist_type == 'cosine':
        return 1 - np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif dist_type == 'euclid':
        return dis.euclidean(a, b)
    elif dist_type == 'pearson':
        return 1 - st.pearsonr(a, b)[0]
    else:
        print('dist_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(dist_type))

sensors = ['accelerometer', 'gyro', 'emg', 'orientation']
sensor_weights = [0.25, 0.25, 0.25, 0.25]

own = ['big_stop', 'come_down', 'come_here', 'come_up', 'flip', 'small_stop']
paper = ['hcwh', 'hcwv', 'hcws',
         'hcch', 'hccv', 'hccs',
         'tcwh', 'tcwv', 'tcws',
         'tcch', 'tccv', 'tccs',
         'lcwh', 'lcwv', 'lcws',
         'lcch', 'lccv', 'lccs']

In [4]:
start_time = time.time()

parameters = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10],
               'metric':[calculate_distance], 'metric_params':[{'dist_type':'cosine'}, {'dist_type':'euclid'},
                                                             {'dist_type':'pearson'}]}]
frame_numbers = list(range(1,13))
cross_valid = 5

## Vergleiche gleiche Gesten

print('Mean values of different similarity measurements\n')

for gesture in own + paper:
    gesture_dicts = load_gestures(glob('../Data/converted/*/' + gesture + '/*'))
    
    print('Gesture: {}'.format(gesture))
    print('-'*20)
    for sensortype in sensors:
        feature_vectors = prepare_data(gesture_dicts, FRAME_NUMBER, sensortype, verbose=False)[0]

        cosine_simi = []
        euclid_simi = []
        pearson_simi = []

        for gesture in combinations(feature_vectors, 2):
            cosine_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='cosine'))
            euclid_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='euclid'))
            pearson_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='pearson'))

        print('Sensor type: {}'.format(sensortype))
        print('Cosine similarity: {}'.format(np.mean(cosine_simi)))
        print('Euclid similarity: {}'.format(np.mean(euclid_simi)))
        print('Pearson similarity: {}'.format(np.mean(pearson_simi)))
        print()
    print()

start_time = time.time()
gesture_dicts = [load_gestures(glob('../Data/converted/*/' + gesture + '/*')) for gesture in own+paper]

prepared_groups = []
for gesture_group in gesture_dicts:
    gesture = []
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'accelerometer', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'gyro', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'emg', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'orientation', verbose=False)[0])
    prepared_groups.append(gesture)
        
print('Duration: {}'.format(time.time() - start_time))

for gesture, gesture_label in zip(combinations_with_replacement(prepared_groups, 2),
                                  combinations_with_replacement(own+paper, 2)):
    sensor_similarity_cosine = []
    sensor_similarity_euclid = []
    sensor_similarity_pearson = []
    
    print('{} to {}'.format(gesture_label[0], gesture_label[1]))
    for sensortype_ga, sensortype_gb in zip(gesture[0], gesture[1]):
        
        mean_a = np.divide(np.sum(sensortype_ga, axis=0), len(sensortype_ga))
        mean_b = np.divide(np.sum(sensortype_gb, axis=0), len(sensortype_gb))
        
        sensor_similarity_cosine.append(calculate_similarity(mean_a, mean_b, sim_type='cosine'))
        sensor_similarity_euclid.append(calculate_similarity(mean_a, mean_b, sim_type='euclid'))
        sensor_similarity_pearson.append(calculate_similarity(mean_a, mean_b, sim_type='pearson'))
        

    result_cosine = np.sum(np.multiply(sensor_similarity_cosine, sensor_weights))
    result_euclid = np.sum(np.multiply(sensor_similarity_euclid, sensor_weights))
    result_pearson = np.sum(np.multiply(sensor_similarity_pearson, sensor_weights))
    
    print('Weighted Cosine similarity: {}'.format(result_cosine))
    print('Weighted Euclid similarity: {}'.format(result_euclid))
    print('Weighted Pearson similarity: {}'.format(result_pearson))
    print()

## k Nearest Neighbour - own

In [5]:
train, valid, test = load_data_for_classification('own')

own_acc = []
own_gyro = []
own_ori = []
own_emg = []

### Accelerometer

In [6]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(acc_train_input, acc_valid_input, axis=0),
                                                                        np.append(acc_train_label, acc_valid_label))
        own_acc.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        own_acc.append((-1,  nr, kNN.best_params_))

### Gyroskop

In [7]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                                        np.append(gyro_train_label, gyro_valid_label))
        own_gyro.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        own_gyro.append((-1,  nr, kNN.best_params_))

### Magnetometer

In [8]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(ori_train_input, ori_valid_input, axis=0),
                                                                        np.append(ori_train_label, ori_valid_label))
        own_ori.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        own_ori.append((-1,  nr, kNN.best_params_))

c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


### EMG

In [9]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(emg_train_input, emg_valid_input, axis=0),
                                                                        np.append(emg_train_label, emg_valid_label))
        own_emg.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        own_acc.append((-1,  nr, kNN.best_params_))

In [10]:
print(sorted(own_acc, reverse=True)[:7], end='\n')
print(sorted(own_gyro, reverse=True)[:7], end='\n')
print(sorted(own_ori, reverse=True)[:7], end='\n')
print(sorted(own_emg, reverse=True)[:7], end='\n')

[(0.8571428571428571, 5, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'pearson'}, 'n_neighbors': 1}), (0.84555984555984554, 6, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.84108527131782951, 7, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.82625482625482627, 4, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.80308880308880304, 3, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.79457364341085268, 8, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.72868217054263562, 9, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'met

## k Nearest Neighbour - paper

In [11]:
train, valid, test = load_data_for_classification('paper')

paper_acc = []
paper_gyro = []
paper_ori = []
paper_emg = []

### Accelerometer

In [12]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(acc_train_input, acc_valid_input, axis=0),
                                                                        np.append(acc_train_label, acc_valid_label))
        paper_acc.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        paper_acc.append((-1,  nr, kNN.best_params_))

### Gyroskop

In [13]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                                        np.append(gyro_train_label, gyro_valid_label))
        paper_gyro.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        paper_gyro.append((-1,  nr, kNN.best_params_))

### Magnetometer

In [14]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(ori_train_input, ori_valid_input, axis=0),
                                                                        np.append(ori_train_label, ori_valid_label))
        paper_ori.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        paper_ori.append((-1,  nr, kNN.best_params_))

c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


### EMG

In [15]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(emg_train_input, emg_valid_input, axis=0),
                                                                        np.append(emg_train_label, emg_valid_label))
        paper_emg.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        paper_emg.append((-1,  nr, kNN.best_params_))

In [16]:
print(sorted(paper_acc, reverse=True)[:7], end='\n')
print(sorted(paper_gyro, reverse=True)[:7], end='\n')
print(sorted(paper_ori, reverse=True)[:7], end='\n')
print(sorted(paper_emg, reverse=True)[:7], end='\n')

[(0.23969072164948454, 5, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 3}), (0.22809278350515463, 7, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.22422680412371135, 3, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 7}), (0.22036082474226804, 6, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.21134020618556701, 8, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.20360824742268041, 4, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.19329896907216496, 9, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'met

## k Nearest Neighbour - all

In [17]:
train, valid, test = load_data_for_classification('all')

all_acc = []
all_gyro = []
all_ori = []
all_emg = []

### Accelerometer

In [18]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(acc_train_input, acc_valid_input, axis=0),
                                                                        np.append(acc_train_label, acc_valid_label))
        all_acc.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        all_acc.append((-1,  nr, kNN.best_params_))

### Gyroskop

In [19]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                                        np.append(gyro_train_label, gyro_valid_label))
        all_gyro.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        all_gyro.append((-1,  nr, kNN.best_params_))

### Magnetometer

In [ ]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(ori_train_input, ori_valid_input, axis=0),
                                                                        np.append(ori_train_label, ori_valid_label))
        all_ori.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        all_ori.append((-1,  nr, kNN.best_params_))

c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


### EMG

In [23]:
for nr in frame_numbers:
    print(nr, end='\r')
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy').fit(
                                                                        np.append(emg_train_input, emg_valid_input, axis=0),
                                                                        np.append(emg_train_label, emg_valid_label))
        all_emg.append((kNN.best_score_, nr, kNN.best_params_))
    except ValueError as err:
        all_emg.append((-1,  nr, kNN.best_params_))

In [24]:
print(sorted(all_acc, reverse=True)[:7], end='\n')
print(sorted(all_gyro, reverse=True)[:7], end='\n')
print(sorted(all_ori, reverse=True)[:7], end='\n')
print(sorted(all_emg, reverse=True)[:7], end='\n')

[(0.36135265700483093, 5, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 3}), (0.34782608695652173, 6, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.34719535783365568, 7, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.33526570048309179, 3, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.33333333333333331, 4, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1}), (0.32108317214700194, 8, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1}), (0.2852998065764023, 9, {'metric': <function calculate_distance at 0x000001E2E5DB5BF8>, 'metr

In [25]:
print('Duration: {}'.format(time.time() - start_time))

Duration: 330205.29078936577
